# Loading

Create a generator function for importing the file

In [1]:
import urllib.request
import lzma
import base64
import re

def get_dependencies():
    localfile, info = urllib.request.urlretrieve("https://github.com/ogirardot/meta-deps/raw/master/pypi-deps.csv.lzma")
    with lzma.open(localfile, mode='r') as file:
        for line in file:
            package, version, deps = line.decode("utf-8").split('\t')
            deps = base64.b64decode(deps).decode("utf-8")
            deps = eval(deps)
            regex = r'\"*([\w\.]+)[><]?\=?.*\"*'
            
            def cleanDep(s):
                try:
                    match = re.search(regex, s)
                    match = re.search(regex, s)
                    s = match.group(0)
                    s = s.replace('"', '')
                    s = re.split(r'<>=', s)[0]
                    return s
                except:
                    return None
            
            deps = [cleanDep(x) for x in deps if cleanDep(x) is not None]
            
            yield package, deps
        
        


Store as dataframe

In [2]:
import pandas as pd
import numpy as np

rows = np.array([[None,None]])
for package, deps in get_dependencies():
    # Packages with no dependencies get blank string
    if deps:
        args= [[package, dep] for dep in deps]
    else:
        args = [[package, None]]
    rows = np.append(rows, args, axis=0)

        
df = pd.DataFrame(rows, columns=["package", "dependency"]).drop_duplicates()
df.head()

,package,dependency
0,None,None
1,0x10c-asm,None
2,2gis,None
3,3to2,None
4,3to2_py3k,None


In [3]:
deps = pd.Series(df.dependency.unique())
packages = pd.Series(df.package.unique())
new_deps = deps[~deps.isin(packages)]
new_df = pd.DataFrame({'package': new_deps, 'dependency': [None]*len(new_deps)})
df = pd.concat([df, new_df])
df = df.dropna(subset=['package'])

In [14]:
from timeit import default_timer, timeit

## Redshift

In Redshift we create a single table that will hold one "edge" per row.

In [9]:
import redshift_connector
conn = redshift_connector.connect(
     host='redshift-python-dep.cantgzv41ter.us-east-1.redshift.amazonaws.com',
     database='deps',
     user='awsuser',
     password='REDACTED'
  )
conn.autocommit = True
cursor = conn.cursor()

Create the table

In [5]:
cursor.execute("DROP TABLE dependencies")
cursor.execute(
"""
    CREATE TABLE dependencies (
        package VARCHAR(150),
        dependency VARCHAR(150) NULL
    )
"""
)

Load the data

In [8]:
from itertools import product

# Time it
start = default_timer()

#cursor.write_dataframe(df, "dependencies")
cursor.executemany("INSERT INTO dependencies VALUES (%s, %s)", df.itertuples(index=False, name=None))

end = default_timer()
redshift_load_time = end - start
redshift_load_time

2338.004424226

# Neptune Graph

Clear existing data from graph

In [9]:
%%gremlin

g.V().drop()

Load in nodes and edges

In [56]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Scope

In [11]:
def load_nodes(g, df):
    for package_name in df.package.unique():
        try:
            g.add_v('package').property(T.id, package_name).as_(package_name).next()
            #print("added package: " + package_name)
        except:
            print(f"Failed to add package {package_name}")

def load_edges(g, df):
    for index, row in df.iterrows():
        if row['dependency'] is not None:
            g.V(row['package']).add_e("depends_on").to(__.V(row['dependency'])).next()
            #print("Added dependency: " + row['package'] + "->" + row['dependency'])



In [12]:
remoteConnStr = 'wss://[NEPTUNE URL]:8182/gremlin'
remoteConn = DriverRemoteConnection(remoteConnStr,'g')

g = Graph().traversal().withRemote(remoteConn)

start = default_timer()

load_nodes(g, df)
load_edges(g, df)

end = default_timer()
remoteConn.close()

neptune_load_time = end - start
neptune_load_time

156.96739027900003

In [13]:
%%gremlin
g.V()
g.E()

# Queries

## Helper Funcs

In [17]:
def timeRedshiftQuery(sqlTxt):
    cursor = conn.cursor()
    start = default_timer()
    cursor.execute(sqlTxt)
    end = default_timer()
    return end - start

def timeNeptuneQuery(f):
    start = default_timer()
    g = Graph().traversal().withRemote(remoteConn)
    end = default_timer()
    return end - start

## Redshift Recursive View
We save a view that has a row for each package and package it depends on either directly or indirectly.

## 1: Which package has the most direct dependencies and how many?

### Redshift

In [20]:
cursor = conn.cursor()
query = """
    SELECT TOP 1 *
    FROM 
    (
        SELECT package, count(*) as "dep_count"
        FROM dependencies
        WHERE dependency is not NULL
        GROUP BY package
    ) DepCounts
    ORDER BY dep_count DESC, package ASC

"""
cursor.execute(query)
cursor.fetchall()


(['Products.CMFPlone', 100],)

In [63]:
timeRedshiftQuery(query)

NameError: name 'f' is not defined

### Neptune

In [45]:
def query(g):
    g.V().hasLabel('package').inE('depends_on'). \
    groupCount().by('package').as_('group'). \
    order(Scope.local). \
        by('values', Order.desc). \
        by('keys', Order.asc). \
    unfold().limit(1).toList()

In [1]:
%%gremlin

g.V().hasLabel('package').in('depends_on').
groupCount().by(id).
order(Scope.local).
    by(values, Order.desc).
    by(keys, Order.asc).
unfold().limit(1)

# 3: Which packages depend on Pandas (directly and indirectly)?

## Redshift

In [23]:
query = """
    WITH RECURSIVE panda_deps(package, dependency, level) AS 
    (SELECT package, dependency, 0 AS level 
     FROM dependencies
     WHERE dependency = 'numpy'
     UNION ALL
     SELECT d.package, d.dependency, level + 1
     FROM dependencies d, panda_deps p
     WHERE d.dependency = p.package
    )
    SELECT package FROM panda_deps;

"""
cursor.execute(query)
cursor.fetchall()

(['ADAM-Tools'],
 ['BiologicalProcessNetworks'],
 ['dcmt'],
 ['django-instakit'],
 ['fastinterval'],
 ['gyroid'],
 ['hdf5-django'],
 ['lmj.c3d'],
 ['lmj.nethack'],
 ['lmj.rbm'],
 ['nplook'],
 ['oceans'],
 ['PyBUFR'],
 ['pyorbital'],
 ['python-consume'],
 ['pyucsc'],
 ['RTM'],
 ['smith'],
 ['zipline'],
 ['pyucsc'],
 ['canvas'],
 ['chebpy'],
 ['fitsio'],
 ['gceising'],
 ['gsw'],
 ['h5dj'],
 ['his2h5'],
 ['linguistic-helper-functions'],
 ['lmj.kohonen'],
 ['lmj.particle'],
 ['lmj.plot'],
 ['lmj.pursuit'],
 ['metaseq'],
 ['multichain_mcmc'],
 ['nwalign'],
 ['ocupy'],
 ['pyRserve'],
 ['readfmf'],
 ['SimpleHist'],
 ['topzootools'])

In [24]:
timeRedshiftQuery(query)

0.0029741880002802645

# 4: Which packages have multiple dependency?


## Redshift

In [20]:
query = """
WITH RECURSIVE total_deps(package, dependency, level) AS 
    (SELECT package, dependency, 0 AS level 
     FROM dependencies
     UNION ALL
     SELECT d.package, d.dependency, level + 1
     FROM dependencies d, total_deps p
     WHERE 
        d.dependency = p.package AND
        level <= 50
    )
SELECT DISTINCT package from
(
    SELECT package, count(dependency) as total_paths
    FROM total_deps
    GROUP BY package, dependency
    HAVING total_paths > 1
) t
"""
cursor.execute(query)
cursor.fetchall()

(['abl.jquery.ui'],
 ['abu.rpc'],
 ['actdiag'],
 ['ADAM-Tools'],
 ['afpy.xap'],
 ['AgileCLU'],
 ['agsci.blognewsletter'],
 ['anthill.skinner'],
 ['archetypes.fieldtraverser'],
 ['archetypes.recurringdate'],
 ['Argot'],
 ['Art3dUtils'],
 ['asm.cms'],
 ['asm.cmsui'],
 ['assentio'],
 ['assert-select'],
 ['atomisator.outputs'],
 ['atreal.massloader'],
 ['Attest'],
 ['AuthBWC'],
 ['aws.authrss'],
 ['aws.windowsplonecluster'],
 ['baanlib'],
 ['baas'],
 ['babble.demo'],
 ['baboon'],
 ['balanced-ach'],
 ['bam2fpkc'],
 ['bamboo'],
 ['batchhttp'],
 ['Batman'],
 ['bda.calendar.base'],
 ['bda.intellidatetime'],
 ['bda.plone.finder'],
 ['BeakerHelpers'],
 ['beanstalk-stack'],
 ['bebop'],
 ['beeswithmachineguns'],
 ['beets'],
 ['benchmaster'],
 ['bibliograph.rendering'],
 ['bingpy'],
 ['bitmat'],
 ['bitsyauth'],
 ['bitsyblog'],
 ['blockdiagcontrib-square'],
 ['blogdown'],
 ['bob.example.faceverify'],
 ['bongo'],
 ['bookreader'],
 ['Boris'],
 ['boto_utils'],
 ['botox'],
 ['bowerrecipe'],
 ['braspag']

In [21]:
timeRedshiftQuery(query)

0.017207170999881782

## Neptune

From each node, traverse out. Store visited nodes, and return if a visited node is found.

In [22]:
%%gremlin
g.V().as('a').
    repeat(aggregate('visited').out()).until(out().as('visited')).
    select('a').dedup()